In [2]:
import pandas as pd

In [4]:
state_pop_df = pd.read_csv("data/state_populations_cleaned.csv")
state_vacc_df = pd.read_csv("data/state_covid_vaccinations_cleaned.csv")

In [5]:
state_pop_df

,state,population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223
5,Colorado,5758736
6,Connecticut,3565287
7,Delaware,973764
8,District of Columbia,705749
9,Florida,21477737


In [6]:
state_vacc_df

,state,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
0,Alaska,154325,71907,21096,9829,59449,8126,12352,1688
1,Alabama,493125,165919,10057,3384,144429,2946,21345,435
2,Arkansas,361550,173312,11981,5743,144792,4798,26541,879
3,Arizona,797550,303899,10957,4175,262120,3601,41613,572
4,California,4379500,1633875,11084,4135,1335886,3381,293834,744
5,Colorado,657250,353194,11413,6133,296816,5154,54474,946
6,Connecticut,435075,264707,12203,7425,230913,6477,28602,802
7,District of Columbia,75175,55108,10652,7808,43377,6146,11685,1656
8,Delaware,106600,49369,10947,5070,41900,4303,7355,755
9,Florida,2546050,1239975,11854,5773,1125338,5240,111763,520
